## Prompt Engineering in LangSmith

### Import environment variables

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path=".env", override=True)
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")

### Pull in Prompt from Prompthub

In [ ]:
from langsmith import Client

client = Client(api_key=LANGSMITH_API_KEY)
prompt = client.pull_prompt("eli5-concise", include_model=True)

### Setup AI Application

Let's first setup our web search tool, as usual

In [ ]:
# Initialize web search tool

from langchain_community.tools.tavily_search import TavilySearchResults

web_search_tool = TavilySearchResults(max_results=1)

Let's now create our application, same as in the tracing module. This time, our prompt is the one pulled from PromptHub

In [ ]:
from openai import OpenAI
from langsmith import traceable
from langsmith.wrappers import wrap_openai

# Create Application
openai_client = wrap_openai(OpenAI())

@traceable
def search(question):
    web_docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in web_docs])
    return web_results
    
@traceable
def explain(question, context):
    result = prompt.invoke({"question": question, "context": context})
    return result.content

@traceable
def eli5(question):
    context = search(question)
    answer = explain(question, context)
    return answer

### Test Application

In [ ]:
question = "what is complexity economics?"
print(eli5(question))